; Chat Demo - Fixed Query Demonstration
; =====================================
; This file runs a fixed set of queries to demonstrate the ReAct agent.
; For interactive use, see chat.lisp instead.
;
; Usage (interactive):
;   cd src
;   acl2
;   (ld "chat-demo.lisp")
;
; Usage (certification):
;   cert.pl chat-demo --acl2-cmd 'env SKIP_INTERACTIVE=1 acl2'
;   or just: cert.pl chat-demo  (uses cert_env below)

In [1]:
(in-package "ACL2")

 "ACL2"


In [2]:
(include-book "chat-lib"
              :ttags ((:quicklisp) (:quicklisp.osicat) (:quicklisp.dexador) 
                      (:http-json) (:llm-client) (:mcp-client)))


TTAG NOTE (for included book): Adding ttag :QUICKLISP from book /home/acl2/books/quicklisp/base.

ACL2 Warning [Ttags] in ( INCLUDE-BOOK "http-json" ...):  The ttag
note just printed to the terminal indicates a modification to ACL2.
To avoid this warning, supply an explicit :TTAGS argument when including
the book "/workspaces/verified-agent/src/http-json".


TTAG NOTE (for included book): Adding ttag :QUICKLISP.DEXADOR from book /home/acl2/books/quicklisp/dexador.

ACL2 Warning [Ttags] in ( INCLUDE-BOOK "http-json" ...):  The ttag
note just printed to the terminal indicates a modification to ACL2.
To avoid this warning, supply an explicit :TTAGS argument when including
the book "/workspaces/verified-agent/src/http-json".


TTAG NOTE (for included book): Adding ttag :HTTP-JSON from book /workspaces/verified-agent/src/http-json.

ACL2 Warning [Ttags] in ( INCLUDE-BOOK "http-json" ...):  The ttag
note just printed to the terminal indicates a modification to ACL2.
To avoid this warning, s

In [3]:
#-skip-interactive
(make-event
 (mv-let (err models state)
   (llm-list-models-full state)
   (prog2$ 
    (if err
        (cw "~%Error getting models: ~s0~%" err)
      (progn$
       (cw "~%Available models: ~x0~%" (len models))
       (cw "Loaded completions models: ~x0~%" 
           (len (filter-loaded-completions-models models)))))
    (mv nil '(value-triple :models-listed) state))))


Available models: 15
Loaded completions models: 0

Summary
Form:  ( MAKE-EVENT (MV-LET ...))
Rules: NIL
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
 :MODELS-LISTED


In [4]:
#-skip-interactive
(make-event
 (mv-let (err models state)
   (llm-list-models-full state)
   (declare (ignorable err))
   (let ((selected (select-completions-model models *model-prefs*)))
     (prog2$
      (if selected
          (cw "~%Selected model: ~s0 (context: ~x1 tokens)~%"
              (model-info->id selected)
              (model-info->loaded-context-length selected))
        (cw "~%WARNING: No suitable model found! Make sure LM Studio is running.~%"))
      (mv nil 
          `(defconst *model-id* 
             ,(if selected (model-info->id selected) "no-model-found"))
          state)))))



Summary
Form:  ( DEFCONST *MODEL-ID* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)

Summary
Form:  ( MAKE-EVENT (MV-LET ...))
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *MODEL-ID*


In [5]:
(defconst *demo-agent-v0*
  (init-agent-conversation *default-system-prompt* *default-agent-config*))


Summary
Form:  ( DEFCONST *DEMO-AGENT-V0* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *DEMO-AGENT-V0*


In [6]:
#-skip-interactive
(defconst *check-init*
  (prog2$ (cw "~%Agent initialized. Messages: ~x0~%" 
              (len (get-messages *demo-agent-v0*)))
          t))


Agent initialized. Messages: 1

Summary
Form:  ( DEFCONST *CHECK-INIT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *CHECK-INIT*


In [7]:
#-skip-interactive
(defconst *demo-agent-v1*
  (add-user-msg "Hello! Can you explain what a formally verified agent is?" 
                *demo-agent-v0*))


Summary
Form:  ( DEFCONST *DEMO-AGENT-V1* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *DEMO-AGENT-V1*


In [8]:
#-skip-interactive
(defconst *show-v1*
  (show-conversation *demo-agent-v1*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified ReAct
agent framework built in ACL2.

You have access to ACL2 code execution. To run ACL2 code, put it in
a fenced code block:

```acl2
(+ 1 2 3)
```

or:

```lisp
(defun factorial (n) (if (zp n) 1 (* n (factorial (1- n)))))
```

I will execute the code and show you the result. You can then continue
reasoning or give a final answer.

Be concise. Show your reasoning.

[USER]
Hello! Can you explain what a formally verified agent is?


Summary
Form:  ( DEFCONST *SHOW-V1* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *SHOW-V1*


In [9]:
#-skip-interactive
(make-event
 (mv-let (err response state)
   (llm-chat-completion *model-id* (get-messages *demo-agent-v1*) state)
   (if err
       (prog2$ (cw "~%LLM Error: ~s0~%" err)
               (mv nil `(defconst *demo-agent-v2* *demo-agent-v1*) state))
     (prog2$ (cw "~%Assistant: ~s0~%" response)
             (mv nil `(defconst *demo-agent-v2* 
                        (add-assistant-msg ,response *demo-agent-v1*)) 
                 state)))))


LLM Error: An HTTP request to "http://host.docker.internal:1234/v1/chat/comple\
tions" returned 400 bad request.

#(123 10 32 32 32 32 34 101 114 114 111 114 34 58 32 123 10 32 32 32
32 32 32
  32 32 34 109 101 115 115 97 103 101 34 58 32 34 78 111 32 109 111
100 101 108
  115 32 108 111 97 100 101 100 46 32 80 108 101 97 115 101 32 108
111 97 100
  32 97 32 109 111 100 101 108 32 105 110 32 116 104 101 32 100 101
118 101 108
  111 112 101 114 32 112 97 103 101 32 111 114 32 117 115 101 32 116
104 101 32
  39 108 109 115 32 108 111 97 100 39 32 99 111 109 109 97 110 100
46 34 44 10
  32 32 32 32 32 32 32 32 34 116 121 112 101 34 58 32 34 105 110 118
97 108 105
  100 95 114 101 113 117 101 115 116 95 101 114 114 111 114 34 44 10
32 32 32
  32 32 32 32 32 34 112 97 114 97 109 34 58 32 34 109 111 100 101 108
34 44 10
  32 32 32 32 32 32 32 32 34 99 111 100 101 34 58 32 110 117 108 108
10 32 32
  32 32 125 10 125)

Summary
Form:  ( DEFCONST *DEMO-AGENT-V2* ...)
Rules: NIL
Time:  0.00 seco

In [10]:
#-skip-interactive
(defconst *show-v2*
  (show-conversation *demo-agent-v2*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified ReAct
agent framework built in ACL2.

You have access to ACL2 code execution. To run ACL2 code, put it in
a fenced code block:

```acl2
(+ 1 2 3)
```

or:

```lisp
(defun factorial (n) (if (zp n) 1 (* n (factorial (1- n)))))
```

I will execute the code and show you the result. You can then continue
reasoning or give a final answer.

Be concise. Show your reasoning.

[USER]
Hello! Can you explain what a formally verified agent is?


Summary
Form:  ( DEFCONST *SHOW-V2* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *SHOW-V2*


In [11]:
#-skip-interactive
(defconst *demo-agent-v3*
  (add-user-msg "What properties have been proven about this agent?" 
                *demo-agent-v2*))


Summary
Form:  ( DEFCONST *DEMO-AGENT-V3* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *DEMO-AGENT-V3*


In [12]:
#-skip-interactive
(make-event
 (mv-let (err response state)
   (llm-chat-completion *model-id* (get-messages *demo-agent-v3*) state)
   (if err
       (prog2$ (cw "~%LLM Error: ~s0~%" err)
               (mv nil `(defconst *demo-agent-v4* *demo-agent-v3*) state))
     (prog2$ (cw "~%Assistant: ~s0~%" response)
             (mv nil `(defconst *demo-agent-v4* 
                        (add-assistant-msg ,response *demo-agent-v3*)) 
                 state)))))


LLM Error: An HTTP request to "http://host.docker.internal:1234/v1/chat/comple\
tions" returned 400 bad request.

#(123 10 32 32 32 32 34 101 114 114 111 114 34 58 32 123 10 32 32 32
32 32 32
  32 32 34 109 101 115 115 97 103 101 34 58 32 34 78 111 32 109 111
100 101 108
  115 32 108 111 97 100 101 100 46 32 80 108 101 97 115 101 32 108
111 97 100
  32 97 32 109 111 100 101 108 32 105 110 32 116 104 101 32 100 101
118 101 108
  111 112 101 114 32 112 97 103 101 32 111 114 32 117 115 101 32 116
104 101 32
  39 108 109 115 32 108 111 97 100 39 32 99 111 109 109 97 110 100
46 34 44 10
  32 32 32 32 32 32 32 32 34 116 121 112 101 34 58 32 34 105 110 118
97 108 105
  100 95 114 101 113 117 101 115 116 95 101 114 114 111 114 34 44 10
32 32 32
  32 32 32 32 32 34 112 97 114 97 109 34 58 32 34 109 111 100 101 108
34 44 10
  32 32 32 32 32 32 32 32 34 99 111 100 101 34 58 32 110 117 108 108
10 32 32
  32 32 125 10 125)

Summary
Form:  ( DEFCONST *DEMO-AGENT-V4* ...)
Rules: NIL
Time:  0.00 seco

In [13]:
#-skip-interactive
(defconst *show-v4*
  (show-conversation *demo-agent-v4*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified ReAct
agent framework built in ACL2.

You have access to ACL2 code execution. To run ACL2 code, put it in
a fenced code block:

```acl2
(+ 1 2 3)
```

or:

```lisp
(defun factorial (n) (if (zp n) 1 (* n (factorial (1- n)))))
```

I will execute the code and show you the result. You can then continue
reasoning or give a final answer.

Be concise. Show your reasoning.

[USER]
Hello! Can you explain what a formally verified agent is?

[USER]
What properties have been proven about this agent?


Summary
Form:  ( DEFCONST *SHOW-V4* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *SHOW-V4*


In [14]:
#-skip-interactive
(defconst *demo-agent-v5*
  (add-user-msg "That's impressive! How does the context management work?" 
                *demo-agent-v4*))


Summary
Form:  ( DEFCONST *DEMO-AGENT-V5* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *DEMO-AGENT-V5*


In [15]:
#-skip-interactive
(make-event
 (mv-let (err response state)
   (llm-chat-completion *model-id* (get-messages *demo-agent-v5*) state)
   (if err
       (prog2$ (cw "~%LLM Error: ~s0~%" err)
               (mv nil `(defconst *demo-agent-v6* *demo-agent-v5*) state))
     (prog2$ (cw "~%Assistant: ~s0~%" response)
             (mv nil `(defconst *demo-agent-v6* 
                        (add-assistant-msg ,response *demo-agent-v5*)) 
                 state)))))


LLM Error: An HTTP request to "http://host.docker.internal:1234/v1/chat/comple\
tions" returned 400 bad request.

#(123 10 32 32 32 32 34 101 114 114 111 114 34 58 32 123 10 32 32 32
32 32 32
  32 32 34 109 101 115 115 97 103 101 34 58 32 34 78 111 32 109 111
100 101 108
  115 32 108 111 97 100 101 100 46 32 80 108 101 97 115 101 32 108
111 97 100
  32 97 32 109 111 100 101 108 32 105 110 32 116 104 101 32 100 101
118 101 108
  111 112 101 114 32 112 97 103 101 32 111 114 32 117 115 101 32 116
104 101 32
  39 108 109 115 32 108 111 97 100 39 32 99 111 109 109 97 110 100
46 34 44 10
  32 32 32 32 32 32 32 32 34 116 121 112 101 34 58 32 34 105 110 118
97 108 105
  100 95 114 101 113 117 101 115 116 95 101 114 114 111 114 34 44 10
32 32 32
  32 32 32 32 32 34 112 97 114 97 109 34 58 32 34 109 111 100 101 108
34 44 10
  32 32 32 32 32 32 32 32 34 99 111 100 101 34 58 32 110 117 108 108
10 32 32
  32 32 125 10 125)

Summary
Form:  ( DEFCONST *DEMO-AGENT-V6* ...)
Rules: NIL
Time:  0.00 seco

In [16]:
#-skip-interactive
(defconst *show-v6*
  (show-conversation *demo-agent-v6*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified ReAct
agent framework built in ACL2.

You have access to ACL2 code execution. To run ACL2 code, put it in
a fenced code block:

```acl2
(+ 1 2 3)
```

or:

```lisp
(defun factorial (n) (if (zp n) 1 (* n (factorial (1- n)))))
```

I will execute the code and show you the result. You can then continue
reasoning or give a final answer.

Be concise. Show your reasoning.

[USER]
Hello! Can you explain what a formally verified agent is?

[USER]
What properties have been proven about this agent?

[USER]
That's impressive! How does the context management work?


Summary
Form:  ( DEFCONST *SHOW-V6* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *SHOW-V6*


In [17]:
#-skip-interactive
(defconst *context-check*
  (show-context-usage *demo-agent-v6*))


Context Usage:
  Total characters: 597
  Estimated tokens: 150
  Max tokens: 8000
  Fits in context: T

Summary
Form:  ( DEFCONST *CONTEXT-CHECK* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *CONTEXT-CHECK*


In [18]:
#-skip-interactive
(defconst *demo-agent-with-tool*
  (add-tool-result 
    "File contents of verified-agent.lisp (first 500 chars):
; Verified ReAct Agent - ACL2 Implementation
; This file implements a formally verified ReAct agent...
(in-package \"ACL2\")
(include-book \"std/util/define\" :dir :system)
..." 
    *demo-agent-v6*))


Summary
Form:  ( DEFCONST *DEMO-AGENT-WITH-TOOL* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *DEMO-AGENT-WITH-TOOL*


In [19]:
#-skip-interactive
(defconst *show-tool*
  (show-conversation *demo-agent-with-tool*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified ReAct
agent framework built in ACL2.

You have access to ACL2 code execution. To run ACL2 code, put it in
a fenced code block:

```acl2
(+ 1 2 3)
```

or:

```lisp
(defun factorial (n) (if (zp n) 1 (* n (factorial (1- n)))))
```

I will execute the code and show you the result. You can then continue
reasoning or give a final answer.

Be concise. Show your reasoning.

[USER]
Hello! Can you explain what a formally verified agent is?

[USER]
What properties have been proven about this agent?

[USER]
That's impressive! How does the context management work?

[TOOL]
File contents of verified-agent.lisp (first 500 chars):   (in-package
"ACL2") (include-book "std/util/define" :dir :system) ...


Summary
Form:  ( DEFCONST *SHOW-TOOL* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *SHOW-TOOL*


In [20]:
(defconst *read-file-tool*
  (make-tool-spec
    :name 'read-file
    :required-access 1
    :requires-execute nil
    :token-cost 100
    :time-cost 5))


Summary
Form:  ( DEFCONST *READ-FILE-TOOL* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *READ-FILE-TOOL*


In [21]:
#-skip-interactive
(defconst *can-read-file*
  (prog2$ (cw "~%Can invoke read-file tool: ~x0~%" 
              (can-invoke-tool-p *read-file-tool* *demo-agent-v6*))
          (can-invoke-tool-p *read-file-tool* *demo-agent-v6*)))


Can invoke read-file tool: T

Summary
Form:  ( DEFCONST *CAN-READ-FILE* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *CAN-READ-FILE*


In [22]:
#-skip-interactive
(defconst *demo-agent-after-react*
  (if (and (not (must-respond-p *demo-agent-v6*))
           (can-invoke-tool-p *read-file-tool* *demo-agent-v6*))
      (react-step-with-conversation
        (agent-action-tool-call 'read-file "verified-agent.lisp")
        *read-file-tool*
        "I'll read the verified-agent.lisp file to show you the implementation."
        "File read successfully. Contents: (in-package ACL2)..."
        *demo-agent-v6*)
    *demo-agent-v6*))


Summary
Form:  ( DEFCONST *DEMO-AGENT-AFTER-REACT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *DEMO-AGENT-AFTER-REACT*


In [23]:
#-skip-interactive
(defconst *show-react*
  (prog2$ (cw "~%After ReAct step:~%")
    (prog2$ (cw "  Step counter: ~x0~%" 
                (agent-state->step-counter *demo-agent-after-react*))
      (prog2$ (cw "  Token budget: ~x0~%" 
                  (agent-state->token-budget *demo-agent-after-react*))
        (cw "  Message count: ~x0~%" 
            (len (get-messages *demo-agent-after-react*)))))))


After ReAct step:
  Step counter: 1
  Token budget: 49900
  Message count: 6

Summary
Form:  ( DEFCONST *SHOW-REACT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *SHOW-REACT*


In [24]:
#-skip-interactive
(defconst *demo-complete*
  (prog2$ (cw "~%~%========================================~%")
    (prog2$ (cw "Chat Demo Complete!~%")
      (prog2$ (cw "========================================~%")
        (prog2$ (cw "~%Key points demonstrated:~%")
          (prog2$ (cw "  1. Agent state initialization~%")
            (prog2$ (cw "  2. Conversation management (add-user-msg, add-assistant-msg)~%")
              (prog2$ (cw "  3. Context length tracking~%")
                (prog2$ (cw "  4. Tool result handling~%")
                  (prog2$ (cw "  5. ReAct step with conversation~%")
                    (prog2$ (cw "~%For interactive chat, use chat.lisp instead:~%")
                      (prog2$ (cw "  (ld \"chat.lisp\")~%")
                        (cw "  (chat \"your message\")~%")))))))))))))



Chat Demo Complete!

Key points demonstrated:
  1. Agent state initialization
  2. Conversation management (add-user-msg, add-assistant-msg)
  3. Context length tracking
  4. Tool result handling
  5. ReAct step with conversation

For interactive chat, use chat.lisp instead:
  (ld "chat.lisp")
  (chat "your message")

Summary
Form:  ( DEFCONST *DEMO-COMPLETE* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *DEMO-COMPLETE*
